# コンテナを利用して Lambda 関数を更新する
このスクリプトを実行するための IAM の権限や、AWS configure とかは事前に要設定

In [ ]:
import json
import pprint

with open('conf.json') as f:
    conf = json.load(f)

pprint.pprint(conf)

In [ ]:
import boto3, datetime, subprocess, json
from time import sleep

from boto3.session import Session
profile = conf["profile"]
session = Session(profile_name=profile)

In [ ]:
tag = conf["tag"]
repository_name = conf["repository_name"]

In [ ]:
domain = conf["domain"]
uri = domain + "/" + repository_name

## ECR にコンテナイメージをプッシュ

In [ ]:
%%time

# build からプッシュまで

# build 済なら実行不要
!docker build -t {repository_name} --platform=linux/amd64 .

# タグ付与
!docker tag {repository_name}{tag} {uri}{tag}

# ECR にログイン
! aws ecr get-login-password --profile {profile} | docker login --username AWS --password-stdin {domain}

# push
!docker push {uri}{tag}

In [ ]:
# 必要な情報を抜き取っておく
ecr = session.client('ecr')
res = ecr.describe_images(
    repositoryName = repository_name
)
image_digest = res['imageDetails'][-1]['imageDigest']

## Lambda を実行するロールを取得

In [ ]:
function_name =conf["function_name"]
role_arn = conf["role_arn"]

## Lambda 関数を更新する

In [ ]:
%%time
# lambda function を Docker で作成したイメージから作成する
lambda_client = session.client('lambda')
res = lambda_client.update_function_code(
    FunctionName=function_name,
    ImageUri= f'{uri}@{image_digest}'
)
# 作成が完了するまで待つ
while True:
    res = lambda_client.get_function(FunctionName=function_name)
    configuration = res['Configuration']
    StateReasonCode = configuration['StateReasonCode'] if 'StateReasonCode' in configuration else None
    LastUpdateStatus = configuration['LastUpdateStatus']
    
    try:
        if StateReasonCode=='Creating' or LastUpdateStatus=='InProgress':
            print('.',end='')
            sleep(1)
        elif LastUpdateStatus=='Successful':
            print('!')
            break
    except:
        if LastUpdateStatus=='Successful':
            print('!')
            break
        else:
            print('?')
            break


## 推論してみる

In [ ]:
import base64
import requests
import cv2
import numpy as np
from matplotlib import pyplot as plt
input_file = './yolov5/data/images/bus.jpg'
data = {}
with open(input_file,'rb') as f:
    data["body"]= base64.b64encode(f.read()).decode('utf-8')

In [ ]:
%%time
res = lambda_client.invoke(
    FunctionName=function_name,
    Payload=json.dumps(data)
)

payload = res["Payload"]

body = payload.read()
body = body.decode()
json_dict = json.loads(body)
print(json_dict)